In [ ]:
## 모델 이어서 학습할 때
import pandas as pd
import re
import os
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
#from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import math
from tqdm.notebook import tqdm

def clean_korean_text(text):
    cleaned = re.sub(r'[^가-힣\s]', '', str(text))
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()

def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

def rating_to_sentiment(rating):
    try:
        rating = int(rating)
        if rating == 5: return 3
        elif rating == 4: return 2
        elif rating == 2: return 1
        elif rating == 1: return 0
        else: return None
    except:
        return None

def preprocess_naver_shopping_data(file_path):
    try:
        df = pd.read_csv(file_path, sep='\t', header=None, names=['rating', 'review'])
        print(f"총 {len(df)}개의 리뷰 로드 완료")
    except Exception as e:
        print(f"파일 로딩 오류: {e}")
        return None

    print("텍스트 정제 중...")
    df['cleaned_review'] = df['review'].apply(clean_korean_text)
    df = df[df['cleaned_review'].str.len() > 0]

    print("중복 리뷰 제거 중...")
    before_dedup = len(df)
    df = df.drop_duplicates(subset=['cleaned_review'], keep='first')
    print(f"{before_dedup - len(df)}개 중복 리뷰 제거 완료, {len(df)}개 남음")

    print("불용어 제거 중...")
    stopwords = ['은', '는', '이', '가', '고', '을', '를']
    df['processed_review'] = df['cleaned_review'].apply(lambda x: remove_stopwords(x, stopwords))

    print("감성 라벨 매핑 중...")
    df['sentiment_label'] = df['rating'].apply(rating_to_sentiment)
    df = df.dropna(subset=['sentiment_label'])
    df['sentiment_label'] = df['sentiment_label'].astype(int)

    sentiment_counts = df['sentiment_label'].value_counts().sort_index()
    sentiment_names = {0: '매우 부정적', 1: '부정적', 2: '긍정적', 3: '매우 긍정적'}
    print("\n감성 라벨 분포:")
    for label, count in sentiment_counts.items():
        print(f"  {sentiment_names[label]} ({label}): {count}개")
    return df

class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def evaluate_model(model, data_loader, device):
    """모델 평가 함수"""
    model.eval()
    predictions = []
    true_labels = []
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(true_labels, predictions)
    
    return avg_loss, accuracy, true_labels, predictions

def save_checkpoint(model, tokenizer, optimizer, scheduler, epoch, best_val_acc, patience_counter, checkpoint_dir):
    """체크포인트 저장 함수 (스케줄러 포함)"""
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    # 모델과 토크나이저 저장
    model.save_pretrained(checkpoint_dir)
    tokenizer.save_pretrained(checkpoint_dir)
    
    # 학습 상태를 torch.save로 저장 (스케줄러 포함)
    checkpoint_data = {
        'epoch': epoch,
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'best_val_acc': best_val_acc,
        'patience_counter': patience_counter
    }
    
    torch.save(checkpoint_data, os.path.join(checkpoint_dir, 'training_state.pth'))
    print(f"체크포인트가 '{checkpoint_dir}'에 저장되었습니다.")

def load_checkpoint(checkpoint_dir, device, total_steps=None):
    """체크포인트 로드 함수 (스케줄러 포함)"""
    try:
        # 모델과 토크나이저 로드
        model = BertForSequenceClassification.from_pretrained(checkpoint_dir)
        tokenizer = BertTokenizer.from_pretrained(checkpoint_dir)
        
        # 학습 상태를 torch.load로 로드
        checkpoint_data = torch.load(os.path.join(checkpoint_dir, 'training_state.pth'), map_location=device, weights_only=False)

        
        model.to(device)
        optimizer = AdamW(model.parameters(), lr=2e-5)  # 계속 학습시 낮은 learning rate
        optimizer.load_state_dict(checkpoint_data['optimizer_state_dict'])
        
        # 스케줄러 로드
        scheduler = None
        if checkpoint_data.get('scheduler_state_dict') and total_steps:
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
            )
            scheduler.load_state_dict(checkpoint_data['scheduler_state_dict'])
        
        # optimizer state 내 텐서들을 현재 디바이스로 이동
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(device)
        
        print(f"체크포인트 로드 완료: 에폭 {checkpoint_data['epoch']}부터 재개")
        
        return (model, tokenizer, optimizer, scheduler,
                checkpoint_data['epoch'], 
                checkpoint_data['best_val_acc'], 
                checkpoint_data['patience_counter'])
    
    except Exception as e:
        print(f"체크포인트 로드 실패: {e}")
        return None

def save_data_split(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, save_path):
    """데이터 분할 결과 저장 (torch.save 사용)"""
    data_split = {
        'train_texts': train_texts,
        'train_labels': train_labels,
        'val_texts': val_texts,
        'val_labels': val_labels,
        'test_texts': test_texts,
        'test_labels': test_labels
    }
    
    torch.save(data_split, save_path)
    print(f"데이터 분할 결과가 '{save_path}'에 저장되었습니다.")

def load_data_split(save_path):
    """저장된 데이터 분할 결과 로드 (torch.load 사용)"""
    try:
        data_split = torch.load(save_path, map_location='cpu')  # 데이터는 CPU에 로드
        print(f"저장된 데이터 분할을 '{save_path}'에서 로드했습니다.")
        return (data_split['train_texts'], data_split['train_labels'],
                data_split['val_texts'], data_split['val_labels'],
                data_split['test_texts'], data_split['test_labels'])
    except Exception as e:
        print(f"저장된 데이터 분할 로드 실패: {e}")
        return None

# ===================================================================
# 메인 실행부 (체크포인트 지원 버전)
# ===================================================================
if __name__ == "__main__":
    # --- 🚀 학습 설정 (필요에 따라 수정하세요) ---
    USE_SCHEDULER = True
    INITIAL_LR = 5e-5
    CONTINUE_LR = 2e-5
    batch_size = 32
    # 체크포인트 관련 설정
    USE_CHECKPOINT = True  # True: 체크포인트 사용, False: 새로 시작
    CHECKPOINT_DIR = '/kaggle/input/bert/pytorch/epoch10/1/latest_checkpoint'
    DATA_SPLIT_PATH = '/kaggle/input/data-split/data_split.pth'
    # CHECKPOINT_DIR = '/kaggle/working/latest_checkpoint'  # 체크포인트 저장/로드 경로
    # DATA_SPLIT_PATH = '/kaggle/working/data_split.pth'  # 데이터 분할 결과 저장 경로
    
    # 기본 학습 설정 (체크포인트가 없을 때 사용)
    START_FROM_PRETRAINED = True  # True: 사전훈련 모델, False: 기존 학습된 모델
    SAVED_MODEL_PATH = '/kaggle/input/epochs/20epochs'  # 기존 학습된 모델 경로(사용 x)
    PRETRAINED_MODEL_NAME = 'bert-base-multilingual-cased'  # 사전훈련 모델명
    
    TARGET_TOTAL_EPOCHS = 30  # 최종 목표 에폭 수
    EPOCHS_PER_SESSION = 20  # 한 세션당 학습할 에폭 수 (캐글 12시간 제한 고려)
    
    PROCESSED_FILE_PATH = '/kaggle/input/reviews/preprocessed_reviews.csv'
    
    # --- 💾 체크포인트 확인 및 로드 ---
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"사용 디바이스: {device}")
    
    checkpoint_loaded = False
    scheduler = None
    
    if USE_CHECKPOINT and os.path.exists(CHECKPOINT_DIR):
        print("🔄 이전 체크포인트를 발견했습니다. 로드를 시도합니다...")
        
        # 스케줄러 사용시 총 스텝 수 계산이 필요하므로 임시로 데이터 로드
        if USE_SCHEDULER:
            temp_df = pd.read_csv(PROCESSED_FILE_PATH)
            temp_df = temp_df.dropna(subset=['processed_review', 'sentiment_label'])
            train_size = int(len(temp_df) * 0.7)  # 학습 데이터 비율
            total_steps = math.ceil(train_size / batch_size) * (TARGET_TOTAL_EPOCHS)  # batch_size=32 가정
        else:
            total_steps = None
            
        checkpoint_result = load_checkpoint(CHECKPOINT_DIR, device, total_steps)
        
        if checkpoint_result is not None:
            model, tokenizer, optimizer, scheduler, last_epoch, best_val_acc, patience_counter = checkpoint_result
            current_epoch = last_epoch + 1  # 다음 에폭부터 시작
            checkpoint_loaded = True
            print(f"✅ 체크포인트 로드 성공! 에폭 {current_epoch}부터 재개합니다.")
            
            # 저장된 데이터 분할 로드
            data_split_result = load_data_split(DATA_SPLIT_PATH)
            if data_split_result is not None:
                train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = data_split_result
                print("✅ 이전 데이터 분할 결과를 사용합니다.")
            else:
                print("❌ 데이터 분할 로드 실패. 새로 분할합니다.")
                checkpoint_loaded = False
    
    # --- 📊 데이터 준비 (체크포인트가 없는 경우) ---
    if not checkpoint_loaded:
        print("🆕 새로운 학습을 시작합니다...")
        
        # 데이터 로드
        print(f"'{PROCESSED_FILE_PATH}'에서 전처리된 데이터를 불러옵니다.")
        processed_df = pd.read_csv(PROCESSED_FILE_PATH)
        processed_df = processed_df.dropna(subset=['processed_review', 'sentiment_label'])
        processed_df['sentiment_label'] = processed_df['sentiment_label'].astype(int)
        print(f"전체 데이터 개수: {len(processed_df)}")
        
        # 데이터 분할
        print("\n데이터를 학습/검증/테스트 세트로 분할합니다...")
        train_texts, temp_texts, train_labels, temp_labels = train_test_split(
            processed_df['processed_review'].tolist(),
            processed_df['sentiment_label'].tolist(),
            test_size=0.3, random_state=42,
            stratify=processed_df['sentiment_label']
        )
        
        val_texts, test_texts, val_labels, test_labels = train_test_split(
            temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
        )
        
        print(f"학습 데이터: {len(train_texts)}개")
        print(f"검증 데이터: {len(val_texts)}개") 
        print(f"테스트 데이터: {len(test_texts)}개")
        
        # 데이터 분할 결과 저장
        save_data_split(train_texts, train_labels, val_texts, val_labels, 
                       test_texts, test_labels, '/kaggle/working/data_split.pth')
        
        # 모델 및 토크나이저 로드
        if START_FROM_PRETRAINED:
            print(f"사전훈련 모델 '{PRETRAINED_MODEL_NAME}'을 로드합니다.")
            tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
            model = BertForSequenceClassification.from_pretrained(
                PRETRAINED_MODEL_NAME, num_labels=4
            )
            optimizer = AdamW(model.parameters(), lr=INITIAL_LR)
        else:
            print(f"기존 학습된 모델 '{SAVED_MODEL_PATH}'을 로드합니다.")
            tokenizer = BertTokenizer.from_pretrained(SAVED_MODEL_PATH)
            model = BertForSequenceClassification.from_pretrained(SAVED_MODEL_PATH)
            optimizer = AdamW(model.parameters(), lr=CONTINUE_LR)
        
        model.to(device)
        
        # 🎯 학습률 스케줄러 설정
        if USE_SCHEDULER:
            train_size = len(train_texts)
            total_steps = math.ceil(train_size / batch_size) * TARGET_TOTAL_EPOCHS  # batch_size=32
            warmup_steps = int(0.1 * total_steps)  # 전체의 10%를 warmup
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
            )
            print(f"스케줄러 설정: 총 {total_steps} 스텝, 워밍업 {warmup_steps} 스텝")
        else:
            scheduler = None
        
        current_epoch = 1
        best_val_acc = 0.0
        patience_counter = 0
    
    # --- 🔤 토큰화 및 데이터로더 준비 ---
    print("\n토큰화 진행 중...")
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)
    
    train_dataset = ReviewDataset(train_encodings, train_labels)
    val_dataset = ReviewDataset(val_encodings, val_labels)
    test_dataset = ReviewDataset(test_encodings, test_labels)
    
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print("토큰화 및 데이터로더 준비 완료!")
    
    # --- 📈 학습 시작 ---
    end_epoch = min(current_epoch + EPOCHS_PER_SESSION - 1, TARGET_TOTAL_EPOCHS)
    print(f"\n=== 모델 학습 시작 (에폭 {current_epoch}~{end_epoch}) ===")
    print(f"최종 목표: {TARGET_TOTAL_EPOCHS} 에폭")
   
    
    patience = 5  # Early stopping patience
    
    for epoch_idx in range(EPOCHS_PER_SESSION):
        if current_epoch > TARGET_TOTAL_EPOCHS:
            print(f"\n🎉 목표 에폭 {TARGET_TOTAL_EPOCHS}에 도달했습니다!")
            break
            
        print(f"\nEpoch {current_epoch}/{TARGET_TOTAL_EPOCHS}")
        
        # 학습
        model.train()
        train_loss = 0
        loop = tqdm(train_loader, desc=f"Training Epoch {current_epoch}", leave=False)
        
        for batch in loop:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()
            
            loss.backward()
            optimizer.step()
            
            # 🎯 스케줄러 스텝 (사용하는 경우)
            if scheduler:
                scheduler.step()
                current_lr = scheduler.get_last_lr()[0]
                loop.set_postfix(loss=loss.item(), lr=f"{current_lr:.2e}")
            else:
                loop.set_postfix(loss=loss.item())
        
        avg_train_loss = train_loss / len(train_loader)
        
        # 검증
        val_loss, val_acc, _, _ = evaluate_model(model, val_loader, device)
        
        print(f"학습 손실: {avg_train_loss:.4f}")
        print(f"검증 손실: {val_loss:.4f}")  
        print(f"검증 정확도: {val_acc:.4f}")
        
        # Best model 업데이트
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            
            best_model_dir = f'/kaggle/working/best_sentiment_model'
            if not os.path.exists(best_model_dir):
                os.makedirs(best_model_dir)
            model.save_pretrained(best_model_dir)
            tokenizer.save_pretrained(best_model_dir)
            print(f"*** 새로운 최고 성능! 모델을 '{best_model_dir}'에 저장했습니다. ***")
        else:
            patience_counter += 1
            print(f"성능 개선 없음. Patience: {patience_counter}/{patience}")
        
        # 체크포인트 저장 (매 에폭마다)
        save_checkpoint(model, tokenizer, optimizer, scheduler, current_epoch, best_val_acc, patience_counter, '/kaggle/working/latest_checkpoint')
        
        # Early stopping 체크
        if patience_counter >= patience:
            print(f"\nEarly stopping! {patience} 에폭 동안 성능 개선이 없었습니다.")
            break
        
        current_epoch += 1
    
    # --- 📊 세션 종료 시 상태 출력 ---
    print(f"\n=== 현재 세션 완료 ===")
    print(f"완료된 에폭: {current_epoch-1}/{TARGET_TOTAL_EPOCHS}")
    print(f"최고 검증 정확도: {best_val_acc:.4f}")
    
    if current_epoch <= TARGET_TOTAL_EPOCHS:
        print(f"\n⏰ 다음 세션에서 에폭 {current_epoch}부터 재개하세요!")
        print("🔧 다음 세션 실행 시 설정:")
        print("   USE_CHECKPOINT = True")
        print(f"   CHECKPOINT_DIR = '{CHECKPOINT_DIR}'")
    else:
        print("\n🎉 모든 학습이 완료되었습니다!")
        
        # 최종 테스트 평가
        print("\n=== 최종 테스트 세트 평가 ===")
        best_model = BertForSequenceClassification.from_pretrained('/kaggle/working/best_sentiment_model')
        best_model.to(device)
        
        test_loss, test_acc, test_true, test_pred = evaluate_model(best_model, test_loader, device)
        print(f"테스트 손실: {test_loss:.4f}")
        print(f"테스트 정확도: {test_acc:.4f}")
        
        # 상세한 분류 성능 리포트
        print("\n=== 상세 분류 성능 리포트 ===")
        target_names = ['매우 부정적', '부정적', '긍정적', '매우 긍정적']
        print(classification_report(test_true, test_pred, target_names=target_names))
    
    print("\n✅ 프로그램 종료")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import os

# --------------------------------------------------------------------------
# 1. 평가에 필요한 클래스와 함수 정의 (기존 코드에서 가져옴)
# --------------------------------------------------------------------------

class ReviewDataset(Dataset):
    """PyTorch를 위한 커스텀 데이터셋 클래스"""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # 모든 인코딩 값을 텐서로 변환
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # 레이블도 텐서로 변환
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def evaluate_model(model, data_loader, device):
    """모델의 성능을 평가하고 예측 결과와 실제 값을 반환하는 함수"""
    model.eval()  # 모델을 평가 모드로 설정
    
    predictions = []
    true_labels = []
    total_loss = 0
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for batch in tqdm(data_loader, desc="Evaluating"):
            # 데이터를 디바이스로 이동
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # 모델 실행
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            
            # 손실과 예측 결과 계산
            loss = outputs.loss
            total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=-1)
            
            # 결과 저장
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(true_labels, predictions)
    
    return avg_loss, accuracy, true_labels, predictions

def load_data_split(save_path):
    """저장된 데이터 분할 결과를 로드하는 함수"""
    try:
        data_split = torch.load(save_path, map_location='cpu')
        print(f"✅ '{save_path}'에서 데이터 분할 로드 완료.")
        return (data_split['train_texts'], data_split['train_labels'],
                data_split['val_texts'], data_split['val_labels'],
                data_split['test_texts'], data_split['test_labels'])
    except Exception as e:
        print(f"❌ 데이터 분할 로드 실패: {e}")
        return None

# --------------------------------------------------------------------------
# 2. 메인 실행부
# --------------------------------------------------------------------------

# 🚨 **경로를 자신의 환경에 맞게 수정하세요!** 🚨
# 가장 성능이 좋았던 모델 경로
MODEL_PATH = '/kaggle/input/best_bert/pytorch/best_bert/1/best_sentiment_model'
# 학습 시 분할해두었던 데이터 경로
DATA_SPLIT_PATH = '/kaggle/input/data-split/data_split.pth'

# --- 장치 설정 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

# --- 모델 및 토크나이저 로드 ---
if not os.path.exists(MODEL_PATH):
    print(f"오류: '{MODEL_PATH}' 경로에 모델이 없습니다. 경로를 확인해주세요.")
else:
    print(f"'{MODEL_PATH}'에서 모델과 토크나이저를 불러옵니다...")
    model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
    tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
    model.to(device)
    print("모델 로딩 완료.")

    # --- 데이터 로드 및 준비 ---
    if not os.path.exists(DATA_SPLIT_PATH):
        print(f"오류: '{DATA_SPLIT_PATH}' 경로에 데이터 파일이 없습니다. 경로를 확인해주세요.")
    else:
        # 이전에 분할해 둔 데이터 로드 (학습/검증 데이터는 필요 없지만 test 데이터를 위해 로드)
        _, _, _, _, test_texts, test_labels = load_data_split(DATA_SPLIT_PATH)

        # 테스트 데이터 토큰화
        print("테스트 데이터 토큰화 중...")
        test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

        # 데이터셋 및 데이터로더 생성
        test_dataset = ReviewDataset(test_encodings, test_labels)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        print("데이터 준비 완료.")

        # --- 모델 평가 실행 ---
        print("\n🚀 모델 성능 평가를 시작합니다.")
        test_loss, test_acc, test_true, test_pred = evaluate_model(model, test_loader, device)

        # --- 최종 결과 출력 ---
        print("\n📊 최종 평가 결과")
        print("="*30)
        print(f"테스트 손실 (Test Loss): {test_loss:.4f}")
        print(f"테스트 정확도 (Test Accuracy): {test_acc:.4f}")
        print("="*30)

        print("\n📋 상세 분류 성능 리포트 (Classification Report)")
        target_names = ['매우 부정적 (0)', '부정적 (1)', '긍정적 (2)', '매우 긍정적 (3)']
        print(classification_report(test_true, test_pred, target_names=target_names, digits=4))